# DS/CMPSC 410 Fall 2024
## Instructor: Professor John Yen
## 
# Lab 2: MapReduce in Spark

## The goals of this lab are for you to be able to
## - Implement MapReduce using map, flatMap, and reduceByKey in Spark
## - Apply the above for calculating total counts of words in a textfile.
## - Be able to use .take to show contents of an RDD.
## - Be able to generate an error message due to a typo in the input file.
## Total Number of Exercises: 
- Exercise 1: 5 points
- Exercise 2: 5 points
- Exercise 3: 5 points
- Exercise 4: 10 points
- Exercise 5: 10 points
- Exercise 6: 10 points
- Exercise 7: 5 points
## Total Points: 50 points

# Due: midnight, September 8, 2024

# The first thing we need to do in each Jupyter Notebook running pyspark is to import pyspark first.
# Note: You should have followed the instruction of Lab 2 (word document) to install PySpark in your ICDS account.  If not, complete all steps in the instructions before you proceed.

In [1]:
import pyspark

## Once we import pyspark, we need to import an important object called "SparkContext".  
- Note: Every spark program needs a SparkContext object, which provides critical information (e.g., whether this Spark session runs in local mode or cluster mode) for the run-time environment of the Spark session.

In [2]:
from pyspark import SparkContext

## We then create a Spark Context variable.  Once we have a spark context variable, we can execute spark codes. 
- The first parameter specifies that this Spark session is running in "local" mode. 
- The second parameter specifies a name for your Spark application.

### In creating the Spark Context variable below, we specified that this spark code is running in a 
`local`
### mode, with a name
`Lab2`.
### After you run the cell below, be patient to wait for its completion before you "run" the next cell. When the left of the cell shows 
`[*]:`, 
### it means the 'run' is not completed yet. The completion of running a cell is indicated by a number in the brackets such as
`[3]:`.
### Note: You MUST wait for a cell to complete before you 'run' the next cell in PySpark Jupyter notebook. Otherwise, the execution of your later cells may generate errors because the inputs they need are not available yet (e.g., been not been generated by a previous PySpark statement that is still running).

In [3]:
sc=SparkContext("local", "Lab2")
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 16:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=Lab2>

### Note: You MUST wait for a cell to complete before you 'run" the next cell in PySpark Jupyter notebook. Otherwise, the execution of your later cells may generate errors because the inputs they need are not available yet (e.g., have not been generated by a previous PySpark statement that is still running).

In [4]:
sc.setLogLevel("WARN")

## Exercise 1 (5 points) (a) Add your name below AND (b) replace the path below with the path of your home directory (i.e., replace juy1 with your PSU Access ID).
## Answer for Exercise 1
- a: Your Name: Daniel Woodford

In [11]:
text_RDD = sc.textFile("StayingSafeAtPennStateFall2024.txt")
text_RDD

StayingSafeAtPennStateFall2024.txt MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

# Part A: Parsing Input Text into Tokens

# RDD
- RDD is the primary distributed data structure used by Spark for storing/accessing big data in a cluster.  
- We will talk more about RDD next week. For now, we will view RDD as a big table.
- When we use `textFile` of Spark to read an input file, it returns an RDD (a big table), where each entry in the big table is a line of the input file.
- The contents of RDD can be obtained using `.take()` method.

# Take
- Take is an action, which we will discuss more next week.
- Applying `take` method on an RDD shows the content of the RDD.  
- The parameter of take is the number of entries in the RDD to be accessed.

# Exercise 2 (5%)
Complete the code below to show the first five lines of the input text.

In [12]:
text_RDD.take(5)

['',
 '  Penn State <https://www.psu.edu/>',
 '',
 '  * Hotline <https://universityethics.psu.edu/hotline>',
 '  * Give <https://raise.psu.edu/?']

# Map
- Map of Spark applies a function to an input RDD. We often use a `lambda` expression to describe an unnamed function as the parameter for map. 
- The body of the lambda expression below uses Python `strip` and `split` method (for string).
- The Python `strip` method for strings removes spaces at the beginning and at the end of a string.
- The Python `split` method for strings split the string using the specified delimiter (which is space `" "` in this case).
- Returns an `RDD` (a "big table") where each row is the result of applying the map function to a corresponding row of the input RDD.

In [13]:
line_tokens_RDD = text_RDD.map(lambda line: line.strip().split(" "))
line_tokens_RDD

PythonRDD[8] at RDD at PythonRDD.scala:53

# Exercise 3 (5%)
Complete the code below to show the first ten entries of the line_tokens_RDD.

In [14]:
line_tokens_RDD.take(10)

[[''],
 ['Penn', 'State', '<https://www.psu.edu/>'],
 [''],
 ['*', 'Hotline', '<https://universityethics.psu.edu/hotline>'],
 ['*', 'Give', '<https://raise.psu.edu/?'],
 ['utm_source=psumain&utm_medium=referral&utm_campaign=DonateButton-'],
 ['MainNavigation>'],
 ['*', 'Apply', '<https://www.psu.edu/admission/>'],
 [''],
 ['*',
  'This',
  'is',
  'Penn',
  'State',
  '<https://www.psu.edu/this-is-penn-state/>']]

## Observations:
- Tokens/words in each entry in the text_RDD (i.e., each line in the input text file) are splitted/separated by the space character.
- The `lambda` function returns a list of tokens for each line of the text_RDD.

# flatMap 
The Spark `flatMap` method for RDD returns an RDD that removes the boundary between entries/rows of the input RDD big table.
- Applying `flatMap` to `line_tokens_RDD` removes the boundary of different lines in the input text.  In another word, it merges the list of tokens for each lines in the text into a gigantic list of tokens for the entire input document.
- Intuively, the effect of `flatMap` can be understood as **flattening** the internal structures of its input RDD.

## Results of flatMap:
We no longer see the list of tokens in `line_tokens_RDD` the reflects the line structure of the input file.  Instead, each token is an element of this gigantic list: `tokens_RDD`, which we will use to calculate the total number of occurance for each word/token.

In [15]:
tokens_RDD = line_tokens_RDD.flatMap(lambda x: x)
tokens_RDD.take(10)

['',
 'Penn',
 'State',
 '<https://www.psu.edu/>',
 '',
 '*',
 'Hotline',
 '<https://universityethics.psu.edu/hotline>',
 '*',
 'Give']

## Exercise 4 (10%)
- (a) Describe, based on the cotents of tokens_RDD shown by the previous statement, the effect of flatMap. (5%)
- (b) What is the key difference between the output of map and flatmap ?(5%)

## Answers to Exercise 4:
1. If we think of our original map as a 2D grid, flattening reduces the dimensionality to 1D by placing each token that is not on index 0 to a new line. For example, instead of ['Penn', 'State'] being one row, it is now ['Penn',
 'State']

2. The output is now one dimensional with flatmap instead of two dimensional.

# Part B: Counting Word Frequency using MapReduce

### We want to count the total number of time a word/token occurs in the twitter dataset. We can use the concept of MapReduce to do this in a "scalable" way such that we can do this calculation even if the size of twitter dataset is too large to fit into a computer.

### A MapReduce way to achieve this involves two steps:
### Step 1: map each word into a key value pair 
`(<word>, 1)`
### The key of this key-value pair is the word (in the input RDD); the value of the key-value pair is the number 1.
### Step 2: Use reduceByKey in Spark to aggregate all pairs of the same key into
`(<word>, <count>)`.

# Python Lambda function
The lambda function that is the parameter of `map` is applied to each element of the input RDD (which is each token in the tokens_RDD). Lambda function (in Python) has the format of
`lambda <parameter list>: <function body>`
The value returned by the lambda function is either 
- an explicit `return` statement, or
- the value returned by the last statement in the body of the function.

In the example below, the lambda function returns the key value pair `(<input word>, 1)`
because it is the only (hence also the last) statement in the body of the lambda
expression.


In [16]:
token_1_RDD = tokens_RDD.map(lambda x: (x, 1))
token_1_RDD

PythonRDD[11] at RDD at PythonRDD.scala:53

In [17]:
token_1_RDD.take(10)

[('', 1),
 ('Penn', 1),
 ('State', 1),
 ('<https://www.psu.edu/>', 1),
 ('', 1),
 ('*', 1),
 ('Hotline', 1),
 ('<https://universityethics.psu.edu/hotline>', 1),
 ('*', 1),
 ('Give', 1)]

# reduceByKey

The transformation `reduceByKey` also takes a lambda function. However, this function takes TWO parameters.
- The first parameter `lambda x,y: x+y` is an aggregation function, which has two parameters (x and y in the code below).
- The first parameter of reduceByKey x is a counter (initial value is 0).
- The second parameter of reduceByKey y is the 'value' in input key-value pairs to be aggregated with the counter.  
- Since the 'values' in the input key-value pairs are all 1, this lambda function simplies increment the accumulator for each occurance of the key.  Because reduceByKey aggregates across the entire input RDD, the final value (i.e., accumulator) for each word/token is the total number it occurs in the Twitter dataset.
- The second parameter is the number of partitions used to partition the keys (so that the reduce task can be distributed among multiple reduce workers for scalability). In the cluster mode, the choice of this number should include considerations about the total number of nodes available in the cluster

# Exercise 5 (10 points) Complete the code below by choosing a name for the RDD to be generated by `reduceByKey`.

In [18]:
token_1_RBK = token_1_RDD.reduceByKey(lambda x,y: x+y)
token_1_RBK.take(10)

[('', 143),
 ('Penn', 38),
 ('State', 30),
 ('<https://www.psu.edu/>', 2),
 ('*', 65),
 ('Hotline', 2),
 ('<https://universityethics.psu.edu/hotline>', 1),
 ('Give', 1),
 ('<https://raise.psu.edu/?', 1),
 ('utm_source=psumain&utm_medium=referral&utm_campaign=DonateButton-', 1)]

# Exercise 6 (10 points) Complete the code below by filling in the path of your Tweets word count output.

# saveAsTextFile
This action saves the content of the input RDD in the directory specified.  Even though we add ".txt" extension to the directory (to remind ourself it contains text file, not CSV),
the path is used as a directory to store all partitions of the given RDD (i.e., `token_count_RDD` in this lab). 

## An important requirement for saveAsTextFile is the path for the output files SHOULD NOT EXIST.  

## Debugging Tip #1: 
Before you run saveAsTextFile each time, double check whether the output path used does not already exist.  If it does, either 
- (a) remove the directory, or
- (b) change the output path to one that has not been used.

In [20]:
output_file = "Lab2_output.txt"
token_1_RBK.saveAsTextFile(output_file)

# Exercise 7 (5 points) Modify the path of input file to create a typo in the INPUT FILE NAME.  Execute the code below. 
- You do not need to fix the typo.
- (a) When do you expect to see the first error message before running the code below?
- (b) When does the error message actually show up?

In [21]:
text2_RDD = sc.textFile("notreal.txt")
text2_RDD

notreal.txt MapPartitionsRDD[22] at textFile at NativeMethodAccessorImpl.java:0

In [22]:
line2_RDD = text2_RDD.map(lambda line: line.strip().split(" "))

In [23]:
token2_RDD = line2_RDD.flatMap(lambda x: x)

In [24]:
token2_RDD

PythonRDD[23] at RDD at PythonRDD.scala:53

In [25]:
token2_1_RDD = token2_RDD.map(lambda x: (x, 1))

In [26]:
token2_count_RDD = token2_1_RDD.reduceByKey(lambda x,y: x+y)

Py4JJavaError: An error occurred while calling o136.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/storage/home/dfw5416/Lab2/notreal.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scala:

# Answer to Exercise 7 (10 points):
### Type your answer to Exercise 7 below:
- (a) My guess is going to be the 4th code chunk in that section since you're asking me. Perhaps you can create a map and flatten it out of nothing but printing the lack of anything will cause the error.
- (b) The error showed up at the last code chunk saying that the path does not exist.

In [28]:
sc.stop()

## The `.stop()` method terminates the given SparkContext. 